## Analysis of the FVA results for the Octanoate Base Case 
(limited oxygen and nitrogen uptake, c-source is octanoate and pha is produced)

### setup

In [1]:
import extFunc as ext
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]
import numpy as np
from importlib import import_module
import pandas as pd 
import cobra
import optlang
import copy 
import random

In [ ]:
%%capture
#Load Models if not already in RAM

#nameList = ['iJN1462_GLC_UR10_9.xml' , 'iJN1462_GLC_UR6_3.xml', 'iJN1462_GLN_UR5_1.xml', 'iJN1462_GLC_UR7_3.xml' , 'iJN1462_OCT_UR3_4.xml']
#nameList = ['iJN1462_GLC_UR6_3.xml' , 'iJN1462_OCT_UR3_4.xml']
nameList = ['iJN1462_GLC_UR6_3.xml']

if 'modelDict' not in locals():
    modelDict = ext.ImportFunction(nameList)



if 'glc_oct_comp'  not in locals():
    glc_oct_comp = ext.ModelComparison(modelDict=modelDict)


## FVA analysis


In [ ]:
%%capture
#Base case as in Nogales Paper
model           = modelDict['iJN1462_GLC_UR6_3']
model_bounded   = copy.deepcopy(model)

fv                  = cobra.flux_analysis.flux_variability_analysis(model)
fv["cumulative"]    = abs(fv.maximum) + abs(fv.minimum)


## FVA analysis, Plotting FVA, filterig extreme FVA, influnce of the filtered?


####  Assumption through high default boundaries overal variability artificially strethed ?
####  Test, Measure the overall cumulative variability for the bounded and unbound optimized cases.
#### If a difference is visible after removig the artificially stretched reactions, this would mean that theire was an influence

In [ ]:
filterV = 200

fig, axes = plt.subplots(nrows=1, ncols=3,figsize=(35,10))
fv.plot( y=["maximum", "minimum"],ax=axes[0],
        ylabel="Fluxvariability",
        xlabel="reaction identifier",
        title="FVA of all reactions")
fv[(abs(fv.maximum)<filterV) & (abs(fv.minimum)<filterV)].plot( y=["maximum", "minimum"],
        ax=axes[1],ylim=[-80,80],
        ylabel="Fluxvariability",
        xlabel="reaction identifier",
        title="reactions with fva>200 filtered out")
fv[(abs(fv.maximum)<filterV) & (abs(fv.minimum)<filterV)].plot( y=["cumulative"],
        ax=axes[2],ylim=[0,120],
        ylabel="Fluxvariability",
        xlabel="reaction identifier",
        title="cummulative variability of the filtered reactions")

## save list of filtered for later

list_of_filtered = fv[(abs(fv.maximum)>filterV) | (abs(fv.minimum)>filterV)]
print("length of the filtered array = "+str(len(list_of_filtered)))
#print(list_of_filtered.index)


In [ ]:

bounded_model = copy.deepcopy(model)

for reaction in bounded_model.reactions:
    if abs(reaction.lower_bound)>70:
        reaction.lower_bound= 70*np.sign(reaction.lower_bound)
    if abs(reaction.upper_bound)>70:
        reaction.upper_bound =  70*np.sign(reaction.upper_bound)
        
fv_new_model  = cobra.flux_analysis.flux_variability_analysis(bounded_model)
fv_new_model["cumulative"]    = abs(fv_new_model.maximum) + abs(fv_new_model.minimum)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2)
fv_new_model.plot( y=["maximum", "minimum"],
        ax=axes[0],figsize=(35,15),
        ylabel="Fluxvariability",
        xlabel="reaction identifier",
        title="reactions with fva>200 filtered out")
fv_new_model.plot( y=["cumulative"],
        ax=axes[1],ylim=[0,150],
        ylabel="Fluxvariability",
        xlabel="reaction identifier",
        title="cummulative fva values"
        )


#### Assumption through high default boundaries overal variability artificially strethed ?
#### Test, Measure the overall cumulative variability for the bounded and unbound optimized cases.
#### If a difference is visible after removig the artificially stretched reactions, this would mean that theire was an influence

In [ ]:
filter = list_of_filtered.index

ax1 =   fv[~fv.index.isin(filter)].plot( y=["cumulative"],ylim=[0,120])
fv_new_model[~fv_new_model.index.isin(filter)].plot( y=["cumulative"],ax=ax1,
title = "Comparison of cummulative FVA Values restricted/non-restricted Case ",
xlabel = "reaction Name",
ylabel = "FVA cumulative"
)
ax1.legend(["FVA_cum of non-restricted Model", "FVA_cum of restricted Model, abs(lb/ub)<200"])



In [ ]:
diff=fv[~fv.index.isin(filter)]["cumulative"] - fv_new_model[~fv_new_model.index.isin(filter)]["cumulative"]
diff.sort_values(ascending=False).plot(logy=True,
title = "Differenve in cumulative FVA values for restricted and non restricted case",
xlabel = "reaction Name",
ylabel = "diff(cum_FVA)"
)
nonzero_diff = diff[diff > 0]

print("Number of reactions,for which the result of the FVA changed through reducing the biggest boundaries " + str(len(nonzero_diff)))
print("seems as if a signifant account might be numerical, because of small differences ")

In [ ]:
#bin_array = np.logspace(1e-4,1e+4,9)
axA = fv[~fv.index.isin(filter)].hist(column=["maximum", "minimum"] , log=True, bins=50,figsize=(30,10))
fv_new_model[~fv_new_model.index.isin(filter)].hist(column=["maximum", "minimum"] , log=True,bins=50,ax=axA)


### comparing flux distribution for the filtered and not filtered case

In [ ]:
modelDict2 = {
    "baseCaseGlucose" : model ,
    "boundariesLimited to +/- 200" : bounded_model 
    }
comparisonObjectBoundaedModels = ext.ModelComparison(modelDict=modelDict2)
comparisonObjectBoundaedModels.multiModellSummary(fvaDIr=.99,sortKrit='C-Flux')

## Analysis of the reactions which are filtered

In [ ]:
fl = bounded_model.optimize().fluxes
fl[abs(fl)>5]

In [ ]:
fl = model.optimize().fluxes
fl[list_of_filtered.index].plot.hist(
    title="Flux results of the filtered reactions",
    xlabel="Fluxvalues ")

### Are theire exchange reactions being filtered out 

In [ ]:
list_of_filtered.index = list_of_filtered.index.astype('str')
list_of_filtered[list_of_filtered.index.str.contains('EX')]

## Analysis of FVA of the reaction which are on the Carbon conversion Path

In [ ]:

octanoate_path_reaction_ids =   [ "EX_octa_e","OCTAtex", "FACOAL80t2pp", "ACOAD3f", "RECOAH3", "3HAACOAT80", "RHACOAR80"]
ind_nr                      =   [6, 4 , 1, 3, 2,5,7]
fv_main_carbonpath = fv[fv.index.isin(octanoate_path_reaction_ids)]
fv_main_carbonpath.loc[:,"ind_nr"] = ind_nr
fv_main_carbonpath.sort_values("ind_nr").plot.bar(y=['minimum',])